In [1]:
import pandas as pd
import os
import json

In [139]:
raw_data = []
for line in open('lighttag_annotations2.json', 'r', encoding='utf-8'):
    raw_data.append(json.loads(line))
    annotations_and_examples = []
    relations = []
    #у нас json в одну линию записан, поэтому по сути annannotations_and_examples и relations не обнуляются, и накапливают всю
    #инфу, которую надо
    for d in raw_data:
        for k, v in d.items():
            if k == 'annotations_and_examples':
                annotations_and_examples.append(v)
            if k == 'relations':
                relations.append(v)
                
#we have no relations in our task, so we need not them to append, so we need not the_list


In [140]:
#annotations_and_examples contains:

# content
# metadata
# annotations 
# classifications


In [141]:
raw_data = []
for line in annotations_and_examples[0]:
    raw_data.append((line))
    content = []
    metadata = []
    annotations = []
    classifications = []
    for d in raw_data:
        for k, v in d.items():
            if k == 'content':
                content.append(v)
            if k == 'metadata':
                metadata.append(v)
            if k == 'annotations':
                annotations.append(v)
            if k == 'classifications':
                classifications.append(v)

In [142]:
raw_data = []
for line in annotations[0]:
    raw_data.append((line))
    starts = []
    ends = []
    tags = []
    for d in raw_data:
        for k, v in d.items():
            if k == 'start':
                starts.append(v)
            if k == 'end':
                ends.append(v)
            if k == 'tag':
                tags.append(v)

In [143]:
tagged_content = content.copy()

In [144]:
len(tagged_content) == len(content)

True

In [145]:
list_content = list(tagged_content[0])

for i in range(0, len(list_content)):
    if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
        list_content[i] = 'O'
        
tagged_content[0] = "".join(list_content)

In [146]:
len(tagged_content) == len(content)

True

## Избегаем схлопывания многословных тегов

In [148]:
list_content = list(tagged_content[0])

for i in range(0, len(tags)):
    for j in range(starts[i], ends[i]):
        if content[0][j] == ' ':
            list_content[j] = ' '
        else:
            list_content[j] = tags[i]
        
tagged_content[0] = "".join(list_content)

In [149]:
len(tagged_content) == len(content)

True

## end

In [151]:
import re

In [152]:
tagged_content[0] = re.sub(r'O+', 'O', tagged_content[0])
tagged_content[0] = re.sub(r'ж+', 'ж', tagged_content[0])
tagged_content[0] = re.sub(r'п+', 'п', tagged_content[0])
tagged_content[0] = re.sub(r'д+', 'д', tagged_content[0])
tagged_content[0] = re.sub(r'м+', 'м', tagged_content[0])

In [153]:
len(tagged_content[0].split(' ')) == len(content[0].split(' '))

True

## Разделяем на тексты и теги к текстам

In [155]:
list_of_tags = tagged_content[0].split(' \n#######\n')

In [156]:
list_of_texts = content[0].split(' \n#######\n')

In [157]:
if len(list_of_texts) < len(list_of_tags):
    list_of_tags = list_of_tags[:-1] #пустой элемент убираем
    print(len(list_of_texts))
    print(len(list_of_tags))
    print('\n')
print(len(list_of_texts))
print(len(list_of_tags))

11
11


In [158]:
list_of_text_tokens = []
for i in range(0, len(list_of_texts)):
    list_of_text_tokens.append([list_of_texts[i].split(' ')])

In [159]:
list_of_tags_tokens = []
for i in range(0, len(list_of_tags)):
    list_of_tags_tokens.append([list_of_tags[i].split(' ')])

## Проверка совпадения количество тегов - токенов

In [160]:
for i in range(0, len(list_of_text_tokens)):
    if len(list_of_tags_tokens[i][0])!=len(list_of_text_tokens[i][0]):
        print('idx ', i)
        print(len(list_of_tags_tokens[i][0]))
        print(len(list_of_text_tokens[i][0]))
        print('\n')

In [161]:
#проверяем по-токеново idx-й текст
idx = 7
for i in range(0, len((list_of_text_tokens[idx][0]))):
    if list_of_tags_tokens[idx][0][i] !='O':
        print('слово ',list_of_text_tokens[idx][0][i])
        print('тег ',list_of_tags_tokens[idx][0][i])
        print('\n')

слово  МП
тег  п


слово  Нижегородпассажиравтотранс
тег  п


слово  Нижегородской
тег  м


слово  области
тег  м


слово  России
тег  ж


слово  МП
тег  п


слово  Нижегородпассажиравтотранс
тег  п


слово  с
тег  д


слово  января
тег  д


слово  2014
тег  д


слово  года
тег  д


слово  по
тег  д


слово  декабрь
тег  д


слово  2015
тег  д


слово  Нижнего
тег  м


слово  Новгорода
тег  м


слово  
тег  


слово  Дмитрий
тег  п


слово  Цыганков
тег  п




## Переворачиваем в нормальные теги

In [162]:
final_list_of_tags_tokens = []
for i in range(0, len(list_of_tags_tokens)):
    sublist = []
    for j in range(0, len(list_of_tags_tokens[i][0])):
        if list_of_tags_tokens[i][0][j] == 'п':
            sublist.append('OFF')
        elif list_of_tags_tokens[i][0][j] == 'ж':
            sublist.append('SUF')
        elif list_of_tags_tokens[i][0][j] == 'д':
            sublist.append('DAT')
        elif list_of_tags_tokens[i][0][j] == 'м':
            sublist.append('LOC')
        else:
            sublist.append('O')
    final_list_of_tags_tokens.append(sublist)
    

In [163]:
len(final_list_of_tags_tokens)

11

In [164]:
len(final_list_of_tags_tokens[0])

98

In [165]:
len(list_of_tags_tokens[0][0])

98

## Заносим все в один датафрейм, сохр csv

In [166]:
final = []
for i in range(0, len(list_of_text_tokens)):
    final.append([list_of_text_tokens[i][0], final_list_of_tags_tokens[i]])

In [167]:
len(final)

11

In [168]:
df = pd.DataFrame(final, columns=['text', 'tokens'])
df.head()
#сейчас тут списки токенов, можно потом все завернуть в строку через пробелы, должно сразу все быть хорошо (без добавления/убирания лишних символов и артефактов)

,text,tokens
0,"[Федеральное, налоговое, ведомство, США, по, и...","[O, O, O, SUF, O, O, O, O, O, O, OFF, O, O, O,..."
1,"[Приволжское, следственное, управление, на, тр...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[Уголовное, дело, ,, фигурантом, которого, был...","[O, O, O, O, O, O, O, O, OFF, OFF, OFF, OFF, O..."
3,"[Московская, налоговая, инспекция, проводит, н...","[O, O, O, O, O, O, O, O, O, OFF, O, O, O, O, O..."
4,"[Российское, следствие, объявило, украинского,...","[O, O, O, O, O, OFF, OFF, O, O, O, O, O, SUF, ..."


In [169]:
df.to_csv('Lighttag_parsed_data.csv')